In [2]:
def nondominated_points(X):
    """
    Input: A list X of values
    Output: A list P containing the nondominated points
    """
    P = []  # List of nondominated points

    i = 0  # Outer loop index
    while i < len(X):
        Y = X[i]  # Assign Y to the current element
        fl = 0  # Flag for changes in the inner loop

        j = i + 1  # Inner loop starts from the next element
        while j < len(X):
            if Y <= X[j]:
                X.pop(j)  # Remove X[j] if Y dominates it (Y ≤ X[j])
            elif X[j] <= Y:
                X.pop(i)  # Remove Y if X[j] dominates it (X[j] ≤ Y)
                Y = X[j]  # Update Y to the new value
                fl = 1  # Set flag to indicate a change
            else:
                j += 1  # Move to the next element in the list

        P.append(Y)  # Add Y to the list of nondominated points

        if fl == 0:
            X.pop(i)  # If no changes occurred, remove Y from the list
        else:
            i += 1  # Move to the next element only if there were changes

    return P

# Example usage
X = [10, 15, 8, 7, 5, 9]  # Sample input
P = nondominated_points(X)
print("Nondominated points:", P)


Nondominated points: [9]


In [3]:
def nondominated_points_with_filtration(X):
    """
    Input: A list X of values
    Output: A list P containing the nondominated points, with filtration applied
    """
    P = []  # List of nondominated points

    while len(X) > 1:
        Y = X[0]  # Start with the first element in the list

        i = 1  # Start inner loop from the second element
        while i < len(X):
            if Y <= X[i]:
                X.pop(i)  # Remove X[i] if Y dominates it (Y ≤ X[i])
            elif X[i] <= Y:
                Y = X.pop(i)  # Update Y to X[i] and remove Y from X
            else:
                i += 1  # Move to the next element if neither dominates

        P.append(Y)  # Add Y to the list of nondominated points

        # Filtration step: remove all elements from X such that Y ≤ X[k]
        X = [x for x in X if not (Y <= x)]

    if len(X) == 1:
        P.append(X[0])  # If only one element remains, it's nondominated

    return P

# Example usage
X = [10, 15, 8, 7, 5, 9]  # Sample input
P = nondominated_points_with_filtration(X)
print("Nondominated points with filtration:", P)

Nondominated points with filtration: [5]
